# Recommender Systems

In [1]:
from random import gauss as gs, uniform as uni, seed
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

![Netflix](https://miro.medium.com/max/1400/1*jlQxemlP9Yim_rWTqlCFDQ.png)

![Amazon](images/amazon_recommender.png)

## Agenda

- describe the difference between content-based and collaborative-filtering algorithms
- explain and use the cosine similarity metric
- describe the algorithm of alternating least-squares

## Intro

Recommender systems can be classified along various lines. One fundamental distinction is **content-based** vs. **collaborative-filtering** systems.

To illustrate this, consider two different strategies: (a) I recommend items to you that are *similar to other items* you've used/bought/read/watched; and (b) I recommend items to you that people *similar to you* have used/bought/read/watched. The first is the **content-based strategy**; the second is the **collaborative-filtering strategy**. 

Another distinction drawn is in whether (a) the system uses existing ratings to compute user-user or item-item similarity, or (b) the system uses machine learning techniques to make predictions. Recommenders of the first sort are called **memory-based**; recommenders of the second sort are called **model-based**.

## Content-Based Systems

The basic idea here is to recommend items to a user that are *similar to* items that the user has already enjoyed. Suppose we represent TV shows as rows, where the columns represent various features of these TV shows. These features might be things like the presence of a certain actor or the show fitting into a particular genre etc. We'll just use binary features here, perhaps the result of some one-hot encoding:

In [2]:
tv_shows = np.array([[0, 1, 1, 0, 1, 1, 1],
                   [0, 0, 0, 1, 1, 1, 0],
                   [1, 1, 1, 0, 0, 1, 1],
                   [0, 1, 1, 1, 0, 0, 1]])

tv_shows

array([[0, 1, 1, 0, 1, 1, 1],
       [0, 0, 0, 1, 1, 1, 0],
       [1, 1, 1, 0, 0, 1, 1],
       [0, 1, 1, 1, 0, 0, 1]])

Bob likes the TV Show represented by Row \#1. Which show (row) should we recommend to Bob?

One natural way of measuring the similarity between two vectors is by the **cosine of the angle between them**. Two points near one another in feature space will correspond to vectors that nearly overlap, i.e. vectors that describe a small angle $\theta$. And as $\theta$ decreases, $\cos(\theta)$ *increases*. So we'll be looking for large values of the cosine (which ranges between -1 and 1). We can also think of the cosine between two vectors as the *projection of one vector onto the other*:

![image.png](https://www.oreilly.com/library/view/statistics-for-machine/9781788295758/assets/2b4a7a82-ad4c-4b2a-b808-e423a334de6f.png)

We can use this metric easily if we treat our rows (the items we're comparing for similarity) as vectors: We can calculate the cosine of the angle $\theta$ between two vectors $\vec{a}$ and $\vec{b}$ as follows: $\cos(\theta) = \frac{\vec{a}\cdot\vec{b}}{|\vec{a}||\vec{b}|}$

In [3]:
numerators = np.array([tv_shows[0].dot(tv_show) for tv_show in tv_shows[1:]])

In [4]:
numerators

array([2, 4, 3])

In [5]:
denominators = np.array(
    [(np.linalg.norm(tv_shows[0]) * np.linalg.norm(tv_show)) for tv_show in tv_shows[1:]]
)

In [6]:
denominators

array([3.87298335, 5.        , 4.47213595])

In [7]:
numerators / denominators

array([0.51639778, 0.8       , 0.67082039])

Since the cosine similarity to Row \#1 is highest for Row \#3, we would recommend this TV show.

***

## Collaborative Filtering

Now the idea is to recommend items to a user based on what *similar* users have enjoyed. Suppose we have the following recording of explicit ratings of five items by three users:

In [8]:
users = np.array([[5, 4, 3, 4, 5], [3, 1, 1, 2, 5], [4, 2, 3, 1, 4]])

new_user = np.array([5, 0, 0, 0, 0])
users

array([[5, 4, 3, 4, 5],
       [3, 1, 1, 2, 5],
       [4, 2, 3, 1, 4]])

To which user is `new_user` most similar?

One metric is cosine similarity:

In [9]:
new_user_mag = 5

numerators = np.array([new_user.dot(user) for user in users])

In [10]:
numerators

array([25, 15, 20])

In [11]:
denominators = np.array(
    [(np.linalg.norm(new_user) * np.linalg.norm(user)) for user in users]
)

In [12]:
denominators

array([47.69696007, 31.6227766 , 33.91164992])

In [13]:
numerators / denominators

array([0.52414242, 0.47434165, 0.58976782])

But we could also use another metric, such as Pearson Correlation:

In [14]:
[np.corrcoef(new_user, user)[0, 1] for user in users]

[0.5345224838248488, 0.20044593143431824, 0.5144957554275266]

For more on content-based vs. collaborative systems, see [this Wikipedia article](https://en.wikipedia.org/wiki/Collaborative_filtering) and [this blog post](https://towardsdatascience.com/recommendation-systems-models-and-evaluation-84944a84fb8e). [This post](https://dataconomy.com/2015/03/an-introduction-to-recommendation-engines/) on dataconomy is also useful.

***

## Matrix Factorization

Suppose we start with a matrix $R$ of users and products, where each cell records the ranking the relevant user gave to the relevant product. Very often we'll be able to record this data as a sparse matrix, because many users will not have ranked many items.

Imagine factoring this matrix into a user matrix $P$ and an item matrix $Q^T$: $R = PQ^T$. What would the shapes of $P$ and $Q^T$ be? Clearly $P$ must have as many rows as $R$, which is just the number of users who have given ratings. Similarly, $Q^T$ must have as many columns as $R$, which is just the number of items that have received ratings. We also know that the number of columns of $P$ must match the number of rows of $Q^T$ for the factorization to be possible, but this number could really be anything. In practice this will be a small number, and for reasons that will emerge shortly let's refer to these dimensions as **latent features** of the items in $R$. If $p$ is a row of $P$, i.e. a user-vector, and $q$ is a column of $Q^T$, i.e. an item-vector, then $p$ will record the user's particular weights or *preferences* with respect to the latent features, while $q$ will record how the item ranks with respect to those same latent features. This in turn means that we could predict a user's ranking of a particular item simply by calculating the dot-product of $p$ and $q$! 

If we could effect such a factorization, $R = PQ^T$, then we could calculate *all* predictions, i.e. fill in the gaps in $R$, by solving for $P$ and $Q$.

The isolation of these latent features can be achieved in various ways. But this is at heart a matter of **dimensionality reduction**, and so one way is with the [SVD](https://hackernoon.com/introduction-to-recommender-system-part-1-collaborative-filtering-singular-value-decomposition-44c9659c5e75).

An alternative is to use the method of Alternating Least Squares.

### Alternating Least-Squares (ALS)

ALS recommendation systems are often implemented in Spark architectures because of the appropriateness for distributed computing. ALS systems often involve very large datasets (consider how much data the recommendation engine for NETFLIX must have, for example!), and it is often useful to store them as sparse matrices, which Spark's ML library can handle. In fact, Spark's mllib even includes a "Rating" datatype! ALS is **collaborative** and **model-based**, and is especially useful for working with *implicit* ratings.

We're looking for two matrices (a user matrix and an item matrix) into which we can factor our ratings matrix. We can't of course solve for two matrices at once. But here's what we can do:

Make guesses of the values for $P$ and $Q$. Then hold the values of one *constant* so that we can optimize for the values of the other!

Basically this converts our problem into a familiar *least-squares* problem. See [this page](https://textbooks.math.gatech.edu/ila/least-squares.html) and [this page](https://datasciencemadesimpler.wordpress.com/tag/alternating-least-squares/) for more details, but here's the basic idea:

If we have an equation $Ax = b$ for *non-square* $A$, then we have:

$A^TAx = A^Tb$ <br/>
Thus: <br/>
$x = (A^TA)^{-1}A^Tb$

This $(A^TA)^{-1}A^T$ **is the pseudo-inverse of** $A$. We encountered this before in our whirlwind tour of linear algebra.

In [20]:
np.random.seed(42)

A = np.random.rand(5, 5)
b = np.random.rand(5, 1)

In [16]:
np.linalg.inv(A.T.dot(A)).dot(A.T).dot(b)

array([[-226.17808987],
       [ 218.4875674 ],
       [-362.56506539],
       [ 147.90541497],
       [ 350.145907  ]])

The `numpy` library has a shortcut for this: `numpy.linalg.pinv()`:

In [21]:
np.linalg.pinv(A).dot(b)

array([[-226.17808981],
       [ 218.48756735],
       [-362.56506531],
       [ 147.90541493],
       [ 350.14590691]])

"When we talk about collaborative filtering for recommender systems we want to solve the problem of our original matrix having millions of different dimensions, but our 'tastes' not being nearly as complex. Even if i’ve \[sic\] viewed hundreds of items they might just express a couple of different tastes. Here we can actually use matrix factorization to mathematically reduce the dimensionality of our original 'all users by all items' matrix into something much smaller that represents 'all items by some taste dimensions' and 'all users by some taste dimensions'. These dimensions are called ***latent or hidden features*** and we learn them from our data" ([Medium article: "ALS Implicit Collaborative Filtering"](https://medium.com/radon-dev/als-implicit-collaborative-filtering-5ed653ba39fe)).

#### Simple Example

Suppose Max and Erin have rated five films:

In [22]:
ratings_arr = pd.DataFrame([[0, 1, 0, 0, 4], [0, 0, 0, 5, 0]],\
                           index=['max', 'erin'],
             columns=['film' + str(i) for i in range(1, 6)])
ratings_arr

,film1,film2,film3,film4,film5
max,0,1,0,0,4
erin,0,0,0,5,0


Suppose now that we isolate ten latent features of these films, and that we can capture our users, i.e. the tastes of Max and Erin, according to these features. (We'll just fill out a matrix randomly.)

In [23]:
seed(100)
users = []

for _ in range(2):
    user = []
    for _ in range(10):
        user.append(gs(0, 1))
    users.append(user)
users_arr = np.array(users)

In [24]:
users_arr

array([[ 0.67155333,  0.87331967,  0.20361655, -1.55034921, -0.12059128,
        -1.05927574,  0.38143697, -1.17342904,  0.96637182,  0.53248343],
       [ 2.22041991,  0.68901284,  0.85436449, -0.29504752, -0.62335055,
         1.5917369 ,  0.17920475,  0.60086339,  0.3474319 ,  0.85537186]])

Now we'll make another random matrix that expresses *our items* in terms of these latent features.

In [25]:
seed(100)
items = []

for _ in range(5):
    item = []
    for _ in range(10):
        item.append(gs(0, 1))
    items.append(item)
items_arr = np.array(items)

In [26]:
items_arr

array([[ 0.67155333,  0.87331967,  0.20361655, -1.55034921, -0.12059128,
        -1.05927574,  0.38143697, -1.17342904,  0.96637182,  0.53248343],
       [ 2.22041991,  0.68901284,  0.85436449, -0.29504752, -0.62335055,
         1.5917369 ,  0.17920475,  0.60086339,  0.3474319 ,  0.85537186],
       [-1.80291827, -1.83311295,  0.60911087,  2.4250145 , -2.02233576,
        -0.73382756,  0.24510831, -0.53817586, -0.30637608, -0.41367264],
       [ 1.0027436 ,  0.03558136,  0.19013362, -1.27827915,  0.67648704,
         1.79506722,  0.63054322, -0.37947302, -1.35033057,  0.45576721],
       [ 0.42542416, -0.29962041, -2.48035968, -0.87457154, -1.23050164,
        -1.00629648,  0.1857537 , -1.174924  , -0.33108494,  1.29437514]])

In [27]:
users_arr.shape

(2, 10)

In [29]:
items_arr.shape

(5, 10)

To construct our large users-by-items matrix, we'll simply take the product of our two random matrices.

In [30]:
users_arr.dot(items_arr.T)

array([[ 7.53516384,  1.26783511, -5.21738432,  0.36547776,  3.90803828],
       [ 1.26783511, 10.38970834, -6.10853643,  5.03189793, -1.63817674]])

Now here's where the ALS really kicks in: We'll solve for Max's and Erin's preference vectors by multiplying the pseudo-inverse of the items array by their respective ratings vectors:

In [31]:
max_pref = np.linalg.pinv(items_arr).dot(ratings_arr.loc['max', :])
print(max_pref)
erin_pref = np.linalg.pinv(items_arr).dot(ratings_arr.loc['erin', :])
print(erin_pref)

[ 0.47099573 -0.11214938 -0.98527859  0.09266666 -0.64747458  0.00854642
 -0.11601818  0.09586291 -0.08579953  0.55693307]
[-0.23238655 -0.55138144  0.5415508  -0.71191248  0.27767197  0.81157767
  0.78557579 -1.03623954 -1.25490079  0.02606555]


In [32]:
items_arr.T.shape

(10, 5)

We'll predict (or, in this case, reproduce) Max's and Erin's ratings for films by simply multiplying their preference vectors by the transpose of the items array:

In [33]:
newmax = max_pref.dot(items_arr.T)
newmax

array([1.47104551e-15, 1.00000000e+00, 9.71445147e-16, 8.88178420e-16,
       4.00000000e+00])

This lines up with the ratings with which we began.

In [34]:
newerin = erin_pref.dot(items_arr.T)
newerin

array([ 1.99840144e-15, -3.83026943e-15,  3.10862447e-15,  5.00000000e+00,
        3.88578059e-16])

Ditto!

We'll make a quick error calculation:

In [35]:
guess = np.vstack([newmax, newerin])

err = 0
for i in range(2):
    for j in range(len(ratings_arr.values[i, :])):
        if ratings_arr.values[i, j] != 0:
            err += (ratings_arr.values[i, j] - guess[i, j])**2
print(err)

1.232595164407831e-30


#### Second Example

In [36]:
# Users: m x n (m users)
# Items: r x n (r items)
# Ratings: m x r

In [37]:
# If P = users and Q = items, then we want to approximate R = PQ^T
# Let's generate R.

seed(42)
ratings2 = []
for _ in range(100):
    user = []
    for _ in range(100):
        chance = gs(0, 0.4)
        
        # We'll fill our ratings matrix mostly with 0's to represent
        # unrated films. This is NOT standard; we're doing this only
        # to illustrate the general algorithm.
        if chance > 0.5:
            user.append(int(uni(1, 6)))
        else:
            user.append(0)
        
        # This 'if' will simply ensure that everyone has given at least
        # one rating.
        if user.count(0) == 10:
            user[int(uni(0, 10))] = int(uni(1, 6))
    ratings2.append(user)
ratings_arr2 = np.array(ratings2)

In [39]:
len(ratings_arr2)

100

In [40]:
users2 = []

# Random generation of values for the user matrix
for _ in range(100):
    user = []
    for _ in range(10):
        user.append(gs(0, 1))
    users2.append(user)
users_arr2 = np.array(users2)

In [43]:
users_arr2

array([[-5.96830264e-01, -6.60644169e-01,  5.20090935e-01,
        -1.02332900e+00,  4.84720799e-01,  6.34063499e-01,
        -1.23738203e+00,  2.62804166e+00, -9.80718366e-01,
        -7.58465765e-01],
       [ 6.47152634e-01, -1.28732877e+00,  1.30196059e+00,
        -3.54192410e-01,  1.18282908e+00,  1.11582998e+00,
        -5.99358644e-02, -6.55814218e-01,  2.46163440e-01,
        -7.76689319e-01],
       [ 2.65328306e-01, -1.01459196e+00,  6.18856542e-02,
        -1.34603809e-01, -3.84773388e-02,  5.56442772e-01,
         1.43395593e+00,  2.29737204e-01,  3.14491196e-02,
        -4.03696762e-01],
       [-2.14052609e+00,  4.79318950e-01,  8.41153067e-01,
         1.12199449e+00, -1.26228248e+00, -4.56819541e-01,
         2.53746183e-02, -4.76825406e-01, -1.18486123e-01,
         7.94826506e-01],
       [-2.98006369e-01, -1.05783971e+00,  2.72952589e-01,
         4.47948214e-01,  4.94938380e-01, -1.40072089e-01,
         8.54957479e-01, -9.36771563e-01, -5.91063195e-01,
         6.

In [41]:
items2 = []

# Random generation of values for the item matrix
for _ in range(100):
    item = []
    for _ in range(10):
        item.append(gs(0, 1))
    items2.append(item)
items_arr2 = np.array(items2)

In [44]:
items_arr2

array([[ 1.55152282e+00,  8.79746156e-01,  1.24509006e+00,
         6.64803857e-02, -4.39662187e-01,  2.03228916e-02,
         1.78496004e+00, -3.39799513e-01, -1.85459029e+00,
        -2.48532660e+00],
       [ 6.25239182e-01,  8.20026293e-03,  1.91797629e+00,
         2.13416913e+00, -1.01870863e+00,  1.12995740e+00,
         4.36071762e-01, -8.32971695e-01, -2.58398762e-01,
        -1.25577674e-01],
       [ 6.99972657e-01, -5.58047093e-01,  1.28299039e+00,
         3.20342348e-01,  1.37667558e+00, -1.02389697e-01,
         2.83527086e-01, -5.88738615e-01,  4.80044076e-01,
         8.72344918e-01],
       [ 4.76615373e-01, -1.95327544e-02, -1.15133028e+00,
        -7.65326035e-01, -1.86287306e+00, -3.08112624e-01,
        -7.86933050e-02,  3.57913278e-01, -6.09647168e-01,
        -4.80096311e-01],
       [-9.62794289e-01, -1.21679537e+00,  1.12415183e+00,
         7.75595047e-01, -9.53787381e-02, -2.36683302e-01,
         8.85437887e-01, -1.18460983e+00, -6.18594037e-01,
        -1.

Our first guess at filling in the matrix will simply be the matrix product:

In [45]:
guess = users_arr2.dot(items_arr2.T)

In [46]:
guess.shape == ratings_arr2.shape

True

Let's get a measure of error:

In [47]:
ratings_arr2 - guess

array([[ 0.52571245,  3.72234761,  2.13774926, ..., -4.91599835,
         3.56313166,  0.01902403],
       [ 0.43865091, -2.74523879, -2.05217977, ...,  4.01784861,
        -1.21227392,  0.14894549],
       [ 1.95810768,  1.86653253, -0.61249156, ...,  1.4848292 ,
        -0.33905578, -0.87590285],
       ...,
       [-2.31923919, -3.6868694 ,  3.38464046, ...,  3.74399696,
         3.64776123,  5.97132554],
       [-3.3611196 ,  3.9221067 ,  6.43595586, ..., -3.59392207,
         1.68512129, -3.39969919],
       [ 2.63496248,  3.38475465,  0.69257906, ...,  0.88937176,
         0.04162701,  6.55957347]])

In [48]:
err = (ratings_arr2 - guess)**2

np.sum(err)

115792.62101436341

Pretty terrible! But we started with random numbers and have only done one iteration. Let's see if we can do better:

In [49]:
def als(ratings, users, items, reps=10):
    
    ratings_cols = ratings.T
    for _ in range(reps):
        new_users = []
        for i in range(len(ratings)):
            
            user = LinearRegression(fit_intercept=False)\
            .fit(items, ratings[i]).coef_
            new_users.append(user)
        new_users = np.asarray(new_users)
        
        new_items = []
        for i in range(len(ratings)):
            
            item = LinearRegression(fit_intercept=False)\
            .fit(new_users, ratings_cols[i]).coef_
            new_items.append(item)
        new_items = np.asarray(new_items)
        
        guess = new_users.dot(new_items.T)
        err = 0
        for i in range(len(ratings)):
            for j in range(len(ratings[i])):
                if ratings[i, j] != 0:
                    err += (ratings[i, j] - guess[i, j])**2
        print(err)
        
        items = new_items
        
    return new_users.dot(new_items.T)

We should see our error decrease with more iterations:

In [50]:
als(ratings_arr2, users_arr2, items_arr2)[:5, :]

8447.205902300977
6567.174519509627
6300.416564404081
6193.260494106864
6138.461291808534
6107.216830016897
6088.933039778992
6078.551821329418
6073.09458783347
6070.676388893394


array([[ 6.39680659e-01,  9.66323475e-01, -4.32963499e-01,
        -7.01806452e-01,  2.36700602e-01,  3.64361546e+00,
         6.21691722e-01,  4.01314024e-01,  6.58846915e-02,
         2.33850990e-01,  2.65802362e-01,  3.30895952e-01,
         4.87491177e-01,  1.42471726e+00,  8.73727046e-01,
         5.28683179e-01,  1.02281209e-01, -5.16379126e-01,
         1.79986795e-01,  1.66644403e-01, -2.45481827e-01,
        -6.78917601e-01,  1.20680572e+00,  7.39236783e-01,
         2.67342745e-01, -4.64412575e-01,  5.22958676e-01,
         6.40781118e-01,  2.64918999e-01,  5.96151586e-01,
         1.02490603e+00, -8.52946638e-02,  2.01195616e-01,
         4.31664221e-02,  7.38210476e-01,  2.07233433e-01,
         2.51748134e-01,  7.15876478e-01,  5.01881444e-01,
         4.05729030e-01, -7.07448910e-02, -1.65711888e-01,
         2.97758038e-01,  4.25816619e-01, -3.45557557e-01,
         8.26643449e-01, -6.52703284e-01,  7.96705294e-01,
        -4.29702256e-01, -8.50621380e-02,  5.40750300e-0

#### ALS in `pyspark`

We'll talk about Big Data and Spark soon, but I'll just note here that Spark has a recommendation submodule inside its ml (machine learning) module. Source code for `pyspark`'s version [here](https://spark.apache.org/docs/latest/api/python/_modules/pyspark/ml/recommendation.html).